In [ ]:
import pygrib
import numpy as np
import os

In [ ]:
def read_grib2_files(years, month, day, hour, variable_name, demodir):
    # Initialize an array to store data from each year
    yearly_data = []

    for year in years:
        # Construct the file name based on the specified pattern
        file_name = f'fnl_{year:04d}{month:02d}{day:02d}_{hour:02d}_00.grib2'
        file_path = os.path.join(demodir, file_name)

        # Open the GRIB2 file
        grb = pygrib.open(file_path)

        # Select the variable based on the given name
        var = grb.select(name=variable_name)

        # Append the data to the yearly_data list
        yearly_data.extend([message.values for message in var])

        # Close the GRIB2 file
        grb.close()

    # Convert the list of arrays to a NumPy array for easier manipulation
    yearly_data = np.array(yearly_data)

    return yearly_data

In [ ]:
def average_and_write_grib2(years, month, day, hour, variable_name, demodir):
    # Read data from the specified files
    yearly_data = read_grib2_files(years, month, day, hour, variable_name, demodir)

    # Calculate the average along the time axis (axis=0)
    average_data = np.mean(yearly_data, axis=0)

    # Open the first GRIB2 file to get the messages
    first_year = years[0]
    first_file_name = f'fnl_{first_year:04d}{month:02d}{day:02d}_{hour:02d}_00.grib2'
    first_file_path = os.path.join(demodir, first_file_name)
    first_grb = pygrib.open(first_file_path)
    messages = first_grb.select()
    # Close the first year's GRIB2 file
    first_grb.close()

    # Construct the output file name
    output_file_name = f'avg_{first_year:04d}{month:02d}{day:02d}_{hour:02d}_00.grib2'
    output_file_path = os.path.join(demodir, output_file_name)

    # Open the output GRIB2 file for writing
    grbout = open(output_file_path, 'wb')
    # Write all messages from the first year's file
    for message in messages:
        grbout.write(message.tostring())

    # Write the average data as a new message
    grbout.write(average_data)

In [ ]:

# 使用示例
years = list(range(2014, 2023))  # 2014年到2022年
month = 6
day = 16
hour = 0
variable_name = 'Temperature'
demodir = '/path/to/your/files'  # 替换为你的实际文件路径

average_and_write_grib2(years, month, day, hour, variable_name, demodir)
